In [2]:
!pip install pydantic langchain-teddynote langchain_community langchain_huggingface langchain_openai

from google.colab import output

output.clear()

In [3]:
import os

from google.colab import userdata

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_PROJECT"] = "LangChain 실습"
# Langchain API key 입력
# os.environ["LANGCHAIN_API_KEY"] = userdata.get('langsmith_api_key')

In [4]:
import zipfile

# 데이터 처리 및 분석
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm
import seaborn as sns
from matplotlib import pyplot as plt
import datetime

from sklearn.metrics.pairwise import cosine_similarity

# 모델 저장
import joblib

# To ignore all warnings
import warnings
warnings.filterwarnings('ignore')



# 데이터 불러오기

In [16]:
!pip install chardet

In [20]:
import chardet

rawdata = open('에콰도르_902214.csv', 'rb').read()
result = chardet.detect(rawdata)
charenc = result['encoding']
charenc

'ISO-8859-1'

In [21]:
bl_data_sampled_mx_902213=pd.read_csv('멕시코_902213.csv')
bl_data_sampled_mx_902214=pd.read_csv('멕시코_902214.csv')
bl_data_sampled_ec_902213=pd.read_csv('에콰도르_902213.csv')
bl_data_sampled_ec_902214=pd.read_csv('에콰도르_902214.csv', encoding='ISO-8859-1')

bl_data_sampled=pd.concat([bl_data_sampled_mx_902213, bl_data_sampled_mx_902214, bl_data_sampled_ec_902213, bl_data_sampled_ec_902214], axis=0)

display(bl_data_sampled)

,HIST_SN,ISO_WD2_NAT_CD,HSCD,BASE_DE,IMP_CO_NAME,SUPLY_CO_NAME,EXP_NAT_CD,NAT_NAME,PRDT_DC_VAL,SM_IMP_AMT
0,10373256,MX,902213,20201211,REFACCIONES INDUSTRIALES Y DEL TRANSPORTESA DE CV,"PROCONOR, S.A.",IT,Italia,EXTRAORAL X-RAY SYSTEM WITH ACCESSORIES FOR IT...,28808.11
1,10442782,MX,902213,20201211,COMERCIALIZADORA DE EQUIPOS DIESEL SA DE CV,NaN,FR,Francia,DENTAL X-RAY SYSTEM WITH PARTS AND ACCESSORIES...,50003.46
2,10498327,MX,902213,20201211,SOLUCIONES MEDICAS COMERCIAL S. A. DE C. V.,NaN,FR,Francia,X-RAY EQUIPMENT FOR DENTAL USE,54354.56
3,10498366,MX,902213,20201211,SOLUCIONES MEDICAS COMERCIAL S. A. DE C. V.,NaN,FR,Francia,X-RAY EQUIPMENT FOR DENTAL USE WITH ACCESSORIES,18257.26
4,10626149,MX,902213,20201212,IVOCLAR VIVADENT S.A. DE C.V.,PLANMECA OY,FI,Finlandia,X-RAY UNITS,147151.05
...,...,...,...,...,...,...,...,...,...,...
272,9865327,EC,902214,20210405,INGENIERIA MEDICA Y SSO EINGMEDSSA CIA LTDA,ECORAY CO LTD,KR,KR,MEDICAL EQUIPMENT - ULTRA 100 MEDICAL DIAGNOS,6666.00
273,9875260,EC,902214,20210621,FIBA MEDICAL CIA. LTDA.,RAYCO (SHANGHAI) MEDICAL PRODUCTS COMPANY LIMI...,CN,CN,COMPLETE X-RAY EQUIPMENT AND ITS ACCESSORIES,34121.99
274,9876127,EC,902214,20210722,"ACR EQUIMEDIC EQUIPOS MEDICOS, RESPUESTOS Y SE...",SG HEALTHCARE CO LTD,KR,KR,FIXED X-RAY EQUIPMENT WITH ACCESSORIES,34429.00
275,9909032,EC,902214,20210408,EQUIMEDA S.A.,PIXXGEN CORPORATION,KR,KR,MEDICAL DIAGNOSTIC EQUIPMENT FLAT PANEL DETECT...,7001.00


In [53]:
bl_data_sampled.to_csv('bl_data_sampled_mx_ec.csv', index=False)

In [22]:
bk_data_sampled=pd.read_excel('①국내기업데이터샘플_덱스코윈_xray.xlsx')

display(bk_data_sampled.head())

,NO,company_name,company_id,product_name,product_description,HSCD,바이어 희망 국가,[참고] 바이코리아 상품 URL
0,1,"(주)덱스코윈\n(DEXCOWIN CO.,LTD.)",1138193750,의료용 휴대용 X-ray 장비,"ADX6000FB- 의료용 휴대용 x-ray 장비\n\n1. 용도: 구급차, 응급 ...","902213, 902214",에콰도르\n멕시코,NaN


In [23]:
bk_data_sampled['product_description']='''ADX6000FB - Portable Medical X-Ray Device
Applications:
The device is suitable for use in ambulances, emergency services, home visits, nursing homes, remote villages, small clinics, military hospitals, cruise ships, and more.
Advantages over Fixed X-Ray Systems:
Unlike fixed X-ray systems, this portable device does not require special X-ray facilities or extensive training. It can be used anywhere with ease.
Additional Options:
If portability is not feasible, a stand is available for support.
Certification:
The product is CE-certified.
Product Features:
Portability: As a portable X-ray device, it operates on a battery, ensuring high mobility.
Power Specifications: 80 kV x 5 mA. With slight adjustments to the X-ray DR software settings, it can capture images sufficient for initial diagnoses of the whole body, chest, abdomen, limbs (arms and shoulders), and head.
Battery Performance: Equipped with removable batteries, it can take approximately 300 images outdoors or in areas with limited power using two spare batteries.
Lightweight Design: Weighing only 3.6 kg, it can be handheld or mounted on a simple stand and operated via a wired remote control.
Safety Features: The interior is fully shielded with lead to minimize radiation exposure to users, making it a safe product.
Target Dealers or Markets:
Government healthcare distribution programs (e.g., rural medical support centers)
Ambulances or mobile health screening vehicles
Military applications (e.g., field hospital tents or naval vessels)
Sports teams' medical staff for on-site injury assessments and emergency treatment adjustments
Private hospital ambulances or emergency rooms
Medical aid in underdeveloped regions (e.g., Africa, Southeast Asia), disaster zones, or war zones
Forensic investigations (e.g., smuggling cases, homicide investigations)
Security checks (e.g., detecting bugs, small cameras or recorders, parcel explosives) for police SWAT teams or Interpol
Summary:
This product is applicable across all fields requiring X-rays without the need for additional installations. It is ready to use immediately by simply inserting the battery, making its potential market virtually limitless.'''

In [24]:
bk_data_sampled['product_name']='Portable Medical X-Ray Device'
bk_data_sampled['company_name']='DEXCOWIN CO.,LTD.'

In [25]:
bk_data_sampled

,NO,company_name,company_id,product_name,product_description,HSCD,바이어 희망 국가,[참고] 바이코리아 상품 URL
0,1,"DEXCOWIN CO.,LTD.",1138193750,Portable Medical X-Ray Device,ADX6000FB - Portable Medical X-Ray Device\nApp...,"902213, 902214",에콰도르\n멕시코,NaN


In [54]:
bk_data_sampled.to_csv('DEXCOWIN_9022.csv', index=False)

# Langsmith , LLM API KEY 설정

In [26]:
from langchain_teddynote import logging

# logging.langsmith("KOTRA_BL_data_matching")

In [27]:
from langchain_community.chat_models import ChatPerplexity
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


os.environ["OPENAI_API_KEY"] = userdata.get('KOTRA2_openai_api_key')


# API 사용 rate 제한 설정


In [28]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter_openai = InMemoryRateLimiter(
    requests_per_second=8,  # maximum query per day - 10000 (tier 1)
    check_every_n_seconds=0.1,  # Wake up every 100 ms to check whether allowed to make a request,
    max_bucket_size=10,  # Controls the maximum burst size.
    )

<ipython-input-28-d541afd0af05>:3: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter_openai = InMemoryRateLimiter(


# 사용 모델 설정

In [29]:
# chat_openai = ChatOpenAI(temperature=0,  model="gpt-4o-mini", rate_limiter=rate_limiter_openai)
chat_openai = ChatOpenAI(temperature=0,  model="gpt-4o-mini")

# OpenAI API 이용 결과 (model : chatgpt 4o-mini)

# buyKOREA 설명 처리 프롬프트문

---------------------

+ 업무 배경 설명 (제품명, 상품 설명에 대한 데이터임을 인식)
+ HSCODE 4자리 설명을 추가함 (6자리보다 더 광범위한 개념 인식)
+ 제품명과 상품 정보에 기반한 제품 설명 생성
+ 지시 내용별 예시 제공 (one-shot)

In [30]:
keyword_num=5

class DescriptionSummary(BaseModel):
    name : str = Field(description="product name")
    type : str = Field(description="product type")
    keyword : list[str] = Field(description=f"product keywords, which focus on product's strong point and purpose")
    phrase : list[str] = Field(description=f"product phrases, which focus on product's strong point and purpose")

parser= PydanticOutputParser(pydantic_object=DescriptionSummary)

In [38]:
summary_prompt_buykorea = PromptTemplate.from_template(
    """You are given data about product's description(PRDT_DESC) and product name(PRDT_NAME).
    All products fall under HS Code 902213, 902214, which is a subcategory of 9022.
    The description for 9022 is as follows:
    90.22 Apparatus based on the use of X-rays or of alpha, beta, gamma or other ionising radiations, whether or not for medical, surgical, dental or veterinary uses,
     including radiography or radiotherapy apparatus, X-ray tubes and other X-ray generators, high tension generators, control panels and desks, screens, examination or treatment tables, chairs and the like (+).

    Using this information, complete the following tasks:
    Provide a simple explanation for PRDT_DESC using both PRDT_NAME and PRDT_DESC.
    Use publicly available product descriptions from websites to help.
    If the meaning is unclear, do not force an explanation.

    Example :
    PRDT_DESC : 'FIORESE Pure Hydra Water Cream contains Centella Asiatica and Hyaluronic Acid, effectively delivering hydration and providing instant soothing benefits.
    With Niacinamide and Adenosine, it is a dual-functional cosmetic for brightening and wrinkle care. This cream helps create healthy, smooth, and radiant skin.
    The lightweight, hydrating formula absorbs quickly, delivering moisture deep into the skin with a fresh, non-sticky finish. Suitable for all skin types, especially sensitive, dull, and uneven skin.

    For Sensitive Skin: Gentle, irritation-free formula, suitable even for sensitive skin.

    FIORESE Brand: Fiorese is dedicated to clean beauty, using ingredients sourced from nature. 100% cruelty-free.'

    Output Example : 'The FIORESE Pure Hydra Water Cream is a lightweight, hydrating cream containing Centella Asiatica and Hyaluronic Acid, providing deep moisture and soothing benefits.
    It also includes Niacinamide and Adenosine, making it a dual-functional cosmetic for brightening and wrinkle care.
    The formula absorbs quickly, leaving a fresh, non-sticky finish, and is suitable for all skin types, especially sensitive, dull, and uneven skin.
    The product is gentle and irritation-free, ideal for sensitive skin. Fiorese is committed to clean beauty with cruelty-free, nature-sourced ingredients.'

    PRODUCT DESCRIPTION(PRDT_DESC) : {prdt_desc}
    PRODUCT NAME(PRDT_NAME) : {prdt_name}
    """
)


summary_chain_buykorea = summary_prompt_buykorea | chat_openai

In [39]:
buykorea_preprocessed_dict_openai={}

for index, row in bk_data_sampled.iterrows():
  response_buykorea = summary_chain_buykorea.invoke({"prdt_desc": row['product_description'], "prdt_name": row['product_name']})
  print(response_buykorea.content)
  buykorea_preprocessed_dict_openai[row['product_name']]=[response_buykorea.content]

The Portable Medical X-Ray Device (ADX6000FB) is a versatile and lightweight imaging tool designed for various applications, including use in ambulances, emergency services, home visits, nursing homes, and remote clinics. Unlike traditional fixed X-ray systems, this portable device is easy to operate and does not require specialized facilities or extensive training, making it accessible for immediate use in diverse settings.

Key features include battery operation for high mobility, allowing it to capture approximately 300 images in areas with limited power. It has a power specification of 80 kV x 5 mA and can provide initial diagnostic images of the whole body, chest, abdomen, limbs, and head. The device is CE-certified and designed with safety in mind, featuring lead shielding to minimize radiation exposure.

This product is particularly beneficial for government healthcare programs, military applications, sports medical teams, and forensic investigations, making it suitable for a wi

In [40]:
extract_prompt_buykorea = PromptTemplate.from_template(
    """Extract product type and keywords, phrases from input, based on following rules:
    - Strictly exclude any information about capacity or volume
    - Generate a list of keywords related to product's main feature, ensuring that the number of keywords is exactly {{keyword_num}}. Do not exceed or fall short of this limit under any circumstances.
    - Generate a list of phrases related to product's main feature.

    INPUT EXAMPLE : ''' The FIORESE Pure Hydra Water Cream is a lightweight, hydrating cream containing Centella Asiatica and Hyaluronic Acid, providing deep moisture and soothing benefits.
    It also includes Niacinamide and Adenosine, making it a dual-functional cosmetic for brightening and wrinkle care.
    The formula absorbs quickly, leaving a fresh, non-sticky finish, and is suitable for all skin types, especially sensitive, dull, and uneven skin.
    The product is gentle and irritation-free, ideal for sensitive skin. Fiorese is committed to clean beauty with cruelty-free, nature-sourced ingredients'''

    OUTPUT EXAMPLE : '''
    {{'name' : 'FIORESE Pure Hydra Water Cream',
     'type' : 'Cream',
     'keyword' :  ['Lightweight', 'Hydrating', 'Brightening', 'Wrinkle care', 'Sensitive skin'],
     'phrase' : ['Contains Centella Asiatica and Hyaluronic Acid for deep moisture and soothing benefits',
                 'Dual-functional cosmetic for brightening and wrinkle care with Niacinamide and Adenosine',
                 'Quick absorption with a fresh, non-sticky finish, suitable for all skin types']}}
    '''

    INPUT : {input}
    OUTPUT:{{'name' : '<product name>', 'type': '<product type>', 'keyword': [<keyword1>. <keyword2>, ...], 'phrase' : [<phrase1>. <phrase2>, ...]}}
    """
)

extract_prompt_buykorea= extract_prompt_buykorea.partial(keyword_num=keyword_num)

extract_chain_buykorea = extract_prompt_buykorea | chat_openai | parser

In [41]:
for key, value in buykorea_preprocessed_dict_openai.items():
  response_buykorea = extract_chain_buykorea.invoke({"input": value})
  print(response_buykorea)
  buykorea_preprocessed_dict_openai[key].append(response_buykorea.name)
  buykorea_preprocessed_dict_openai[key].append(response_buykorea.type)
  buykorea_preprocessed_dict_openai[key].append(response_buykorea.keyword)
  buykorea_preprocessed_dict_openai[key].append(response_buykorea.phrase)

name='Portable Medical X-Ray Device (ADX6000FB)' type='Imaging Tool' keyword=['Versatile', 'Lightweight', 'Battery-operated', 'Portable', 'CE-certified'] phrase=['Designed for various applications including ambulances, emergency services, and remote clinics', 'Easy to operate without specialized facilities or extensive training', 'High mobility allows capturing images in areas with limited power', 'Features lead shielding to minimize radiation exposure', 'Suitable for government healthcare programs and military applications']


In [42]:
buykorea_preprocessed_dict_openai

{'Portable Medical X-Ray Device': ['The Portable Medical X-Ray Device (ADX6000FB) is a versatile and lightweight imaging tool designed for various applications, including use in ambulances, emergency services, home visits, nursing homes, and remote clinics. Unlike traditional fixed X-ray systems, this portable device is easy to operate and does not require specialized facilities or extensive training, making it accessible for immediate use in diverse settings.\n\nKey features include battery operation for high mobility, allowing it to capture approximately 300 images in areas with limited power. It has a power specification of 80 kV x 5 mA and can provide initial diagnostic images of the whole body, chest, abdomen, limbs, and head. The device is CE-certified and designed with safety in mind, featuring lead shielding to minimize radiation exposure.\n\nThis product is particularly beneficial for government healthcare programs, military applications, sports medical teams, and forensic inv

In [43]:
buykorea_preprocessed_openai=pd.DataFrame.from_dict(buykorea_preprocessed_dict_openai, orient='index', columns=['description_summary','name', 'type', 'keyword', 'phrase'])
buykorea_preprocessed_openai.reset_index(inplace=True)
buykorea_preprocessed_openai.rename(columns={'index':'product_name'}, inplace=True)
buykorea_preprocessed_openai.to_csv('buykorea_preprocessed_openai.csv', index=False)

display(buykorea_preprocessed_openai.head())

,product_name,description_summary,name,type,keyword,phrase
0,Portable Medical X-Ray Device,The Portable Medical X-Ray Device (ADX6000FB) ...,Portable Medical X-Ray Device (ADX6000FB),Imaging Tool,"[Versatile, Lightweight, Battery-operated, Por...",[Designed for various applications including a...


# BL 데이터 이름 처리 프롬프트문

----------------------------

+ 업무 배경 설명 (수입자, 공급자, 상품 설명이 존재함을 인식)
+ HSCODE 4자리 설명을 추가함 (6자리보다 더 광범위한 개념 인식)
+ 공급자와 상품 이름 정보에 기반한 제품 설명 생성
+ 인터넷 검색으로 추가 정보 검색 지시
+ 의미가 확실치 않은 경우 생성 제한
+ 지시 내용별 예시 제공 (one-shot)

In [44]:
generate_prompt_bldata = PromptTemplate.from_template(
    """You are given data where IMP_CO_NAME imports products (PRDT_DC_VAL) from SUPLY_CO_NAME.
    All products fall under HS Code 902213, 902214, which is a subcategory of 9022.
    The description for 9022 is as follows:
    90.22 Apparatus based on the use of X-rays or of alpha, beta, gamma or other ionising radiations, whether or not for medical, surgical, dental or veterinary uses,
    including radiography or radiotherapy apparatus, X-ray tubes and other X-ray generators, high tension generators, control panels and desks, screens, examination or treatment tables, chairs and the like (+).

    Using this information, complete the following tasks:
    Provide a simple explanation for PRDT_DC_VAL using both SUPLY_CO_NAME and PRDT_DC_VAL.
    Use publicly available product descriptions from websites to help.
    If the meaning is unclear, do not force an explanation.

    Input Example:
    PRDT_DC_VAL: COSMELAN 2 (FACE CREAM)
    SUPLY_CO_NAME: MESOESTETIC, S.L.
    Output Example: 'This product is called COSMELAN 2, a face cream designed to reduce pigmentation and improve skin tone.
    It is often used for treating melasma and other skin discolorations by inhibiting melanin production.'


    PRODUCT DESCRIPTION(PRDT_DC_VAL) : {prdt_desc}
    SUPPLEYER NAME(SUPLY_CO_NAME) : {suply_name}
    """
)

generate_chain_bldata = generate_prompt_bldata | chat_openai

In [45]:
bldata_preprocessed_dict_openai={}

for index, row in tqdm(bl_data_sampled.iterrows()):
  response_bldata = generate_chain_bldata.invoke({"prdt_desc": row['PRDT_DC_VAL'], "suply_name":row['SUPLY_CO_NAME']})
  if index%100==0:
    print(f"\nProduct Name : {row['PRDT_DC_VAL']} \n")
    print(response_bldata.content, '\n')
  bldata_preprocessed_dict_openai[row['PRDT_DC_VAL']]=[response_bldata.content]

1it [00:01,  1.56s/it]


Product Name : EXTRAORAL X-RAY SYSTEM WITH ACCESSORIES FOR ITS CORRECT FUNCTIONING 

This product is an Extraoral X-Ray System, which is a type of imaging equipment used in dental and medical settings to capture images of the teeth, jaws, and surrounding structures. It typically includes various accessories to ensure proper functioning, such as positioning devices and control panels. This system is essential for diagnosing dental issues, planning treatments, and monitoring the health of patients' oral structures. The use of X-rays allows for a non-invasive examination, providing valuable insights for healthcare professionals. 



101it [03:10,  1.68s/it]


Product Name : DENTAL X-RAY SYSTEM 

'This product is called a Dental X-Ray System, which is a specialized apparatus used in dental practices to capture images of a patient's teeth, gums, and surrounding structures. It utilizes X-ray technology to provide detailed views that assist dentists in diagnosing dental issues, planning treatments, and monitoring oral health. The system typically includes components such as an X-ray tube, control panel, and imaging software, ensuring safe and effective imaging for dental examinations.' 



118it [03:38,  1.89s/it]

...

1914it [1:06:27,  1.49s/it]


Product Name : DIGITAL DIAGNOSTIC X-RAY EQUIPMENT 

This product is called DIGITAL DIAGNOSTIC X-RAY EQUIPMENT, which is used for capturing high-quality images of the internal structures of the body using X-ray technology. It is essential in medical diagnostics, allowing healthcare professionals to detect and diagnose various conditions by providing clear images of bones, tissues, and organs. This equipment is crucial in hospitals and clinics for effective patient care and treatment planning. 



2014it [1:09:34,  2.62s/it]


Product Name : REMOTE CONTROLLED X-RAY EQUIPMENT 

This product is called REMOTE CONTROLLED X-RAY EQUIPMENT, which is a sophisticated apparatus used in medical imaging to capture detailed images of the inside of the body using X-rays. The remote control feature allows for enhanced precision and safety during procedures, enabling healthcare professionals to operate the equipment from a distance. This type of equipment is essential in various medical settings for diagnosing conditions and guiding treatment plans. It is supplied by SIEMENS HEALTHCARE GMBH, a leading company in medical technology and healthcare solutions. 



2090it [1:11:56,  2.07s/it]


In [46]:
joblib.dump(bldata_preprocessed_dict_openai, 'bldata_preprocessed_dict_openai.pkl')

['bldata_preprocessed_dict_openai.pkl']

In [47]:
bldata_preprocessed_dict_openai_ext=joblib.load('bldata_preprocessed_dict_openai.pkl')

In [48]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "product name", "title": "Name", "type": "string"}, "type": {"description": "product type", "title": "Type", "type": "string"}, "keyword": {"description": "product keywords, which focus on product\'s strong point and purpose", "items": {"type": "string"}, "title": "Keyword", "type": "array"}, "phrase": {"description": "product phrases, which focus on product\'s strong point and purpose", "items": {"type": "string"}, "title": "Phrase", "type": "array"}}, "required": ["name", "type", "keyword", "ph

In [49]:
extract_prompt_bldata = PromptTemplate.from_template(
    """Extract product type and keywords, phrases from input, based on following rules:
    - Strictly exclude any information about capacity or volume
    - Generate a list of keywords related to product's main feature, ensuring that the number of keywords is exactly {{keyword_num}}. Do not exceed or fall short of this limit under any circumstances.
    - Generate a list of phrases related to product's main feature.

    INPUT EXAMPLE : ''' The FIORESE Pure Hydra Water Cream is a lightweight, hydrating cream containing Centella Asiatica and Hyaluronic Acid, providing deep moisture and soothing benefits.
    It also includes Niacinamide and Adenosine, making it a dual-functional cosmetic for brightening and wrinkle care.
    The formula absorbs quickly, leaving a fresh, non-sticky finish, and is suitable for all skin types, especially sensitive, dull, and uneven skin.
    The product is gentle and irritation-free, ideal for sensitive skin. Fiorese is committed to clean beauty with cruelty-free, nature-sourced ingredients'''

    OUTPUT EXAMPLE : '''
    {{"name" : "FIORESE Pure Hydra Water Cream",
     "type" : "Cream",
     "keyword" :  ["Lightweight", "Hydrating", "Brightening", "Wrinkle care", "Sensitive skin"],
     "phrase" : ["Contains Centella Asiatica and Hyaluronic Acid for deep moisture and soothing benefits",
                 "Dual-functional cosmetic for brightening and wrinkle care with Niacinamide and Adenosine",
                 "Quick absorption with a fresh, non-sticky finish, suitable for all skin types"]}}
    '''

    INPUT : {input}
    FORMAT : {format}
    """
)

extract_prompt_bldata= extract_prompt_bldata.partial(keyword_num=keyword_num)
extract_prompt_bldata= extract_prompt_bldata.partial(format=parser.get_format_instructions())

extract_chain_bldata = extract_prompt_bldata | chat_openai | parser

In [50]:
for i, (key, value) in tqdm(enumerate(bldata_preprocessed_dict_openai_ext.items())):
  try:
    response_bldata = extract_chain_bldata.invoke({"input": value[0]})
    if i%100==0:
      print(response_bldata)
    bldata_preprocessed_dict_openai_ext[key].append(response_bldata.name)
    bldata_preprocessed_dict_openai_ext[key].append(response_bldata.type)
    bldata_preprocessed_dict_openai_ext[key].append(response_bldata.keyword)
    bldata_preprocessed_dict_openai_ext[key].append(response_bldata.phrase)
  except:
    print(f"key : {key}")
    print(f"extract error: {response_bldata}")

1it [00:01,  1.79s/it]

name='Extraoral X-Ray System' type='Imaging Equipment' keyword=['Non-invasive', 'Diagnostic', 'Dental', 'Treatment planning', 'Oral health'] phrase=['Essential for diagnosing dental issues and monitoring oral structures', 'Captures images of teeth, jaws, and surrounding structures', 'Includes accessories for proper functioning like positioning devices and control panels']


101it [03:38,  2.77s/it]

name='General Electric Proteus X-ray Device' type='X-ray Device' keyword=['Advanced imaging', 'Medical diagnostics', 'Radiographic examinations', 'Healthcare', 'Clinical use'] phrase=['Used in medical settings to create images of the inside of the body', 'Helps healthcare professionals diagnose and treat various conditions', 'Known for its advanced imaging capabilities', 'Often utilized in hospitals and clinics for radiographic examinations']


201it [07:22,  2.07s/it]

...

1201it [47:29,  1.69s/it]

name='X-Ray Device for Mammography' type='Device' keyword=['Early detection', 'Mammographic imaging', 'Precision', 'Reliability', 'Breast cancer'] phrase=['Designed for the early detection of breast cancer through mammographic imaging', 'Utilizes X-ray technology to create detailed images of breast tissue', "Essential tool in women's health care known for its precision and reliability"]


1301it [50:53,  1.95s/it]

name='High Voltage Generator' type='Generator' keyword=['High voltage', 'Radiology', 'Radiation therapy', 'Diagnostic imaging', 'Reliability'] phrase=['Essential for various medical and industrial applications, particularly in radiology and radiation therapy', 'Provides necessary high voltage to power X-ray tubes and other radiographic equipment', 'Enables generation of X-rays for diagnostic imaging or treatment purposes', 'Known for supplying advanced medical equipment ensuring efficiency in healthcare settings']


1381it [53:57,  2.34s/it]


In [51]:
joblib.dump(bldata_preprocessed_dict_openai_ext, 'bldata_preprocessed_dict_openai_ext.pkl')

['bldata_preprocessed_dict_openai_ext.pkl']

In [55]:
bldata_preprocessed_openai=pd.DataFrame.from_dict(bldata_preprocessed_dict_openai_ext, orient='index', columns=['product_description','name', 'type', 'keyword', 'phrase'])
bldata_preprocessed_openai.reset_index(inplace=True)
bldata_preprocessed_openai.rename(columns={'index':'product_name'}, inplace=True)
bldata_preprocessed_openai.to_csv('bldata_preprocessed_openai.csv', index=False)

display(bldata_preprocessed_openai)

,product_name,product_description,name,type,keyword,phrase
0,EXTRAORAL X-RAY SYSTEM WITH ACCESSORIES FOR IT...,"This product is an Extraoral X-Ray System, whi...",Extraoral X-Ray System,Imaging Equipment,"[Non-invasive, Diagnostic, Dental, Treatment p...",[Essential for diagnosing dental issues and mo...
1,DENTAL X-RAY SYSTEM WITH PARTS AND ACCESSORIES...,"'This product is a Dental X-Ray System, which ...",Dental X-Ray System,X-Ray System,"[Diagnostic, Imaging, Detailed, Cavities, Infe...","[Captures images of teeth, gums, and surroundi..."
2,X-RAY EQUIPMENT FOR DENTAL USE,This product is called X-RAY EQUIPMENT FOR DEN...,X-RAY EQUIPMENT FOR DENTAL USE,Equipment,"[Dental, Imaging, Diagnosis, Treatment plannin...",[Designed for dental applications to capture d...
3,X-RAY EQUIPMENT FOR DENTAL USE WITH ACCESSORIES,'This product is described as X-ray equipment ...,Dental X-ray Equipment,X-ray Equipment,"[Dental use, Image capture, Diagnosis aid, Tre...",[Specifically designed for dental practices to...
4,X-RAY UNITS,"This product is called X-RAY UNITS, which refe...",X-RAY UNITS,Equipment,"[Medical imaging, Diagnostics, Healthcare, Pat...",[Essential for visualizing internal structures...
...,...,...,...,...,...,...
1376,STATIONARY X-RAYS FOR MINOR SPECIES,"This product is called ""Stationary X-Rays for ...",Stationary X-Rays for Minor Species,X-ray apparatus,"[Veterinary, Diagnostic, Imaging, Detailed, Mi...",[Designed for use on smaller animals in veteri...
1377,OSCAR CLASSIC,"'This product is called OSCAR CLASSIC, which i...",OSCAR CLASSIC,X-ray Imaging System,"[Portable, High-quality imaging, User-friendly...",[Designed for medical and dental applications ...
1378,HIGH FREQUENCY MOBILE ARM SYSTEM,This product is called the High Frequency Mobi...,High Frequency Mobile Arm System,Medical Apparatus,"[High-frequency, Therapeutic, Flexible, Pain r...",[Utilizes high-frequency electromagnetic waves...
1379,OPTIMA XR646 X-RAY,"This product is called OPTIMA XR646 X-RAY, whi...",OPTIMA XR646 X-RAY,X-ray Imaging System,"[Advanced, High-quality imaging, User-friendly...",[Utilizes ionizing radiation to create detaile...


# 수출한 기업은 다르지만, 수출한 품목의 이름은 같은 경우가 있어 불일치 발생
(중복 제거됨)

In [ ]:
# 같은 품목 중 다른 기업이 수출한 이력이 있는 경우, 다른 수출건으로 집계됨

bl_data_sampled.shape

In [ ]:
# 제품 이름에 기반해, 제품 수입 확률이 높은 해외 바이어 매칭이므로
# 중복되는 제품 이름은 하나의 키워드 및 임베딩으로 통일

len(bldata_preprocessed_dict_openai)

# 임베딩

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

# 오픈소스 임베딩 모델 다운로드
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
# embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

In [ ]:
def text_embedding(text):
  embedding_text=embeddings.embed_query(text)
  return np.array(embedding_text).reshape(1,-1)

In [ ]:
bldata_preprocessed_openai['product_description'][0]

#  1. 전체 문장 임베딩

In [ ]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

buykorea_preprocessed_openai['buykorea_preprocessed']=buykorea_preprocessed_openai['description_summary']
bldata_preprocessed_openai['bldata_preprocessed']=bldata_preprocessed_openai['product_description']

# 전체 문장을 임베딩
buykorea_preprocessed_openai['buykorea_embedding']=buykorea_preprocessed_openai['buykorea_preprocessed'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_embedding']=bldata_preprocessed_openai['bldata_preprocessed'].progress_apply(text_embedding)

bldata_sample_len=bldata_preprocessed_openai.shape[0]

In [ ]:
openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_description', 'bldata_product_name', 'bldata_description', 'cosine_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_description': [row['buykorea_preprocessed']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_description' : ["None"]*bldata_sample_len,
                               'cosine_similarity' : [0]*bldata_sample_len})

  buykorea_embedding=row['buykorea_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):

    bldata_embedding=row_bl['bldata_embedding']

    similarity=cosine_similarity(buykorea_embedding, bldata_embedding)
    similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
    similarity_df.loc[row_count, 'bldata_description']=row_bl['bldata_preprocessed']
    similarity_df.loc[row_count, 'cosine_similarity']=similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())

In [ ]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'], group_keys=False, sort=False).apply(lambda x: x.sort_values('cosine_similarity', ascending=False))
openai_similarity_df_sorted.reset_index(drop=True, inplace=True)

In [ ]:
# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.groupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted_v1.csv', index=False)

# 2. 제품 유형 유사도 필터 후 전체 문장 임베딩

In [ ]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

# 제품 유형을 임베딩
buykorea_preprocessed_openai['buykorea_type_embedding']=buykorea_preprocessed_openai['type'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_type_embedding']=bldata_preprocessed_openai['type'].progress_apply(text_embedding)

# 전체 문장을 임베딩
buykorea_preprocessed_openai['buykorea_description_embedding']=buykorea_preprocessed_openai['description_summary'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_description_embedding']=bldata_preprocessed_openai['product_description'].progress_apply(text_embedding)

bldata_sample_len=bldata_preprocessed_openai.shape[0]

In [ ]:
openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_type', 'buykorea_description', 'bldata_product_name',
                                           'bldata_type', 'bldata_description', 'type_cosine_similarity', 'description_cosine_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_type': [row['type']]*bldata_sample_len,
                               'buykorea_description': [row['description_summary']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_type' : ["None"]*bldata_sample_len,
                               'bldata_description' : ["None"]*bldata_sample_len,
                               'type_cosine_similarity' : [0]*bldata_sample_len,
                               'description_cosine_similarity' : [0]*bldata_sample_len})

  buykorea_type_embedding=row['buykorea_type_embedding']
  buykorea_description_embedding=row['buykorea_description_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):

    bldata_type_embedding=row_bl['bldata_type_embedding']
    bldata_description_embedding=row_bl['bldata_description_embedding']

    type_similarity=cosine_similarity(buykorea_type_embedding, bldata_type_embedding)
    description_similarity=cosine_similarity(buykorea_description_embedding, bldata_description_embedding)
    similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
    similarity_df.loc[row_count, 'bldata_type']=row_bl['bldata_preprocessed']
    similarity_df.loc[row_count, 'type_cosine_similarity']=type_similarity
    similarity_df.loc[row_count, 'description_cosine_similarity']=description_similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())


NameError: name 'buykorea_preprocessed_openai' is not defined

In [ ]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'], group_keys=False, sort=False).apply(lambda x: x.sort_values(by=['type_cosine_similarity', 'description_cosine_similarity'], ascending=False))
openai_similarity_df_sorted.reset_index(drop=True, inplace=True)

# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.groupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted_v2.csv', index=False)

# 3. 제품 유형 유사도와 키워드 5개 유사도 순 나열

In [ ]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

# 제품 유형을 임베딩
buykorea_preprocessed_openai['buykorea_type_embedding']=buykorea_preprocessed_openai['type'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_type_embedding']=bldata_preprocessed_openai['type'].progress_apply(text_embedding)

# 키워드로 구성된 문장을 임베딩
buykorea_preprocessed_openai['keyword_sentence']=buykorea_preprocessed_openai['keyword'].progress_apply(lambda x :x.strip('[]'))
bldata_preprocessed_openai['keyword_sentence']=bldata_preprocessed_openai['keyword'].progress_apply(lambda x : x.strip('[]'))

buykorea_preprocessed_openai['buykorea_keyword_embedding']=buykorea_preprocessed_openai['keyword_sentence'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_keyword_embedding']=bldata_preprocessed_openai['keyword_sentence'].progress_apply(text_embedding)

bldata_sample_len=bldata_preprocessed_openai.shape[0]

In [ ]:
openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_type', 'buykorea_keyword', 'bldata_product_name', 'bldata_type', 'bldata_keyword', 'type_cosine_similarity' ,'keyword_cosine_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_type': [row['type']]*bldata_sample_len,
                               'buykorea_keyword': [row['keyword_sentence']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_type' : ["None"]*bldata_sample_len,
                               'bldata_keyword' : ["None"]*bldata_sample_len,
                               'type_cosine_similarity' : [0]*bldata_sample_len,
                               'keyword_cosine_similarity' : [0]*bldata_sample_len})

  buykorea_type_embedding=row['buykorea_type_embedding']
  buykorea_keyword_embedding=row['buykorea_keyword_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):

    bldata_type_embedding=row_bl['bldata_type_embedding']
    bldata_keyword_embedding=row_bl['bldata_keyword_embedding']

    type_similarity=cosine_similarity(buykorea_type_embedding, bldata_type_embedding)
    keyword_similarity=cosine_similarity(buykorea_keyword_embedding, bldata_keyword_embedding)
    similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
    similarity_df.loc[row_count, 'bldata_keyword']=row_bl['keyword_sentence']
    similarity_df.loc[row_count, 'type_cosine_similarity']=type_similarity
    similarity_df.loc[row_count, 'keyword_cosine_similarity']=keyword_similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())


In [ ]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'], group_keys=False, sort=False).apply(lambda x: x.sort_values(by=['type_cosine_similarity', 'keyword_cosine_similarity'], ascending=False))
openai_similarity_df_sorted.reset_index(drop=True, inplace=True)

# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.groupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted_v3.csv', index=False)

# 4. 제품 유형 유사도와 전체 문장 + 핵심 구절 n개 임베딩

In [ ]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

buykorea_preprocessed_openai['phrase_sentence']=buykorea_preprocessed_openai['phrase'].progress_apply(lambda x : x.strip('[]'))
bldata_preprocessed_openai['phrase_sentence']=bldata_preprocessed_openai['phrase'].progress_apply(lambda x : x.strip('[]'))

# 제품 유형 임베딩
buykorea_preprocessed_openai['buykorea_type_embedding']=buykorea_preprocessed_openai['type'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_type_embedding']=bldata_preprocessed_openai['type'].progress_apply(text_embedding)

# 전체 문장 + 구절로 구성된 문장을 임베딩

buykorea_preprocessed_openai['keyword_sentence']=buykorea_preprocessed_openai['keyword'].progress_apply(lambda x : x.strip('[]'))
bldata_preprocessed_openai['keyword_sentence']=bldata_preprocessed_openai['keyword'].progress_apply(lambda x : x.strip('[]'))

buykorea_preprocessed_openai['buykorea_description_phrase_embedding']=buykorea_preprocessed_openai['description_summary']+buykorea_preprocessed_openai['phrase_sentence'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_description_phrase_embedding']=bldata_preprocessed_openai['product_description']+bldata_preprocessed_openai['phrase_sentence'].progress_apply(text_embedding)

bldata_sample_len=bldata_preprocessed_openai.shape[0]

In [ ]:
import re

openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_type', 'buykorea_phrase_description',
                                           'bldata_product_name', 'bldata_type','bldata_phrase_description',
                                           'type_cosine_similarity', 'phrase_description_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_type': [row['type']]*bldata_sample_len,
                               'buykorea_phrase_description': [row['description_summary']+row['phrase_sentence']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_type' : ["None"]*bldata_sample_len,
                               'type_cosine_similarity' : [0]*bldata_sample_len,
                               'phrase_description_cosine_similarity' : [0]*bldata_sample_len,
                              })

  buykorea_type_embedding=row['buykorea_type_embedding']
  buykorea_phrase_description_embedding=row['buykorea_description_phrase_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):
      bldata_type_embedding=row_bl['bldata_type_embedding']
      bldata_phrase_description_embedding=row_bl['bldata_description_phrase_embedding']

      type_similarity=cosine_similarity(buykorea_type_embedding, bldata_type_embedding)
      phrase_description_similarity=cosine_similarity(buykorea_phrase_description_embedding, bldata_phrase_description_embedding)
      similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
      similarity_df.loc[row_count, 'bldata_type']=row_bl['type']
      similarity_df.loc[row_count, 'type_cosine_similarity']=type_similarity
      similarity_df.loc[row_count, 'phrase_description_cosine_similarity']=phrase_description_similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())



In [ ]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'],
                                                          group_keys=False, sort=False).apply(lambda x: x.sort_values(by=['type_cosine_similarity','phrase_description_cosine_similarity'], ascending=False))

openai_similarity_df_sorted.reset_index(drop=True, inplace=True)

# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.grozupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted_v4.csv', index=False)

# 5. 제품 유형 유사도와 핵심 구절 n개 임베딩 + 전체 문장 (순서 변화)

In [ ]:
tqdm.pandas()

# 임베딩할 텍스트

buykorea_preprocessed_openai=pd.read_csv('./buykorea_preprocessed_openai.csv')
bldata_preprocessed_openai=pd.read_csv('./bldata_preprocessed_openai.csv')

buykorea_preprocessed_openai['buykorea_preprocessed']=buykorea_preprocessed_openai['description_summary']
bldata_preprocessed_openai['bldata_preprocessed']=bldata_preprocessed_openai['product_description']

# 키워드로 구성된 문장을 임베딩
buykorea_preprocessed_openai['buykorea_embedding']=buykorea_preprocessed_openai['buykorea_preprocessed'].progress_apply(text_embedding)
bldata_preprocessed_openai['bldata_embedding']=bldata_preprocessed_openai['bldata_preprocessed'].progress_apply(text_embedding)


bldata_sample_len=bldata_preprocessed_openai.shape[0]

In [ ]:
openai_similarity_df=pd.DataFrame(columns=['buykorea_product_name', 'buykorea_description', 'bldata_product_name', 'bldata_description', 'cosine_similarity'])

for index, row in buykorea_preprocessed_openai.iterrows():

  # 각 buykorea 상품 (10개) 에,
  # 2800개의 수출 기록 존재하는 제품 이름과의 유사도 계산
  # buykorea 상품별로 2800개의 유사도가 계산되어 저장된 데이터프레임 생성

  similarity_df=pd.DataFrame({'buykorea_product_name':[row['product_name']]*bldata_sample_len,
                               'buykorea_description': [row['buykorea_preprocessed']]*bldata_sample_len,
                               'bldata_product_name' : ["None"]*bldata_sample_len,
                               'bldata_description' : ["None"]*bldata_sample_len,
                               'cosine_similarity' : [0]*bldata_sample_len})

  buykorea_embedding=row['buykorea_embedding']
  row_count=0

  for index, row_bl in tqdm(bldata_preprocessed_openai.iterrows()):

    bldata_embedding=row_bl['bldata_embedding']

    similarity=cosine_similarity(buykorea_embedding, bldata_embedding)
    similarity_df.loc[row_count, 'bldata_product_name']=row_bl['product_name']
    similarity_df.loc[row_count, 'bldata_description']=row_bl['bldata_preprocessed']
    similarity_df.loc[row_count, 'cosine_similarity']=similarity

    row_count+=1

  # buykorea 제품 유사도 데이터프레임을 하나의 데이터프레임으로 병합
  openai_similarity_df=pd.concat([openai_similarity_df, similarity_df], axis=0)

display(openai_similarity_df.head())


In [ ]:
openai_similarity_df_sorted= openai_similarity_df.groupby(['buykorea_product_name'], group_keys=False, sort=False).apply(lambda x: x.sort_values(by=['cosine_similarity'], ascending=False))
openai_similarity_df_sorted.reset_index(drop=True, inplace=True)

# 각 buykorea_product_name 별로 cosine_similarity가 높은 10개, 낮은 10개를 'Highest'로, 나머지는 'Lowest'으로 표시하는 함수
def mark_top_bottom_10(df):
    # 상위 10개와 하위 10개는 'Highest', 나머지는 'Lowest'
    df['test_row'] = ['Highest'] * 10 + ['N'] * (len(df) - 20) + ['Lowest'] * 10
    return df

# 각 buykorea_product_name별로 그룹화하여 처리
# Apply the function and then reset the index to align with the original DataFrame
openai_similarity_df_sorted['test_row'] = openai_similarity_df_sorted.groupby('buykorea_product_name', group_keys=False).apply(mark_top_bottom_10).reset_index(drop=True)['test_row']


# 결과 확인
display(openai_similarity_df_sorted)

openai_similarity_df_sorted.to_csv('openai_similarity_df_sorted_v6.csv', index=False)

# PlantUML 로 추출한 제품 분류 이용

In [ ]:
keyword_num=5

class DescriptionSummary(BaseModel):
    name : str = Field(description="product name")
    type : str = Field(description="product type")
    keyword : list[str] = Field(description=f"product keywords, which focus on product's strong point and purpose")
    phrase : list[str] = Field(description=f"product phrases, which focus on product's strong point and purpose")

parser= PydanticOutputParser(pydantic_object=DescriptionSummary)

In [ ]:
plantUML_extract_prompt_bldata = PromptTemplate.from_template(
    """Extract product type and keywords, phrases from input, based on following rules:
    - Strictly exclude any information about capacity or volume
    - Generate a list of keywords related to product's main feature, ensuring that the number of keywords is exactly {{keyword_num}}. Do not exceed or fall short of this limit under any circumstances.
    - Generate a list of phrases related to product's main feature.
    - If selecting an appropriate type is impossible given the information, output type as 'Not Classifiable'
    - If the input contains multiple types of products, list them in a comma-separated list.

    INPUT EXAMPLE : ''' The FIORESE Pure Hydra Water Cream is a lightweight, hydrating cream containing Centella Asiatica and Hyaluronic Acid, providing deep moisture and soothing benefits.
    It also includes Niacinamide and Adenosine, making it a dual-functional cosmetic for brightening and wrinkle care.
    The formula absorbs quickly, leaving a fresh, non-sticky finish, and is suitable for all skin types, especially sensitive, dull, and uneven skin.
    The product is gentle and irritation-free, ideal for sensitive skin. Fiorese is committed to clean beauty with cruelty-free, nature-sourced ingredients'''

    OUTPUT EXAMPLE : '''
    {{"name" : "FIORESE Pure Hydra Water Cream",
     "type" : "Cream",
     "keyword" :  ["Lightweight", "Hydrating", "Brightening", "Wrinkle care", "Sensitive skin"],
     "phrase" : ["Contains Centella Asiatica and Hyaluronic Acid for deep moisture and soothing benefits",
                 "Dual-functional cosmetic for brightening and wrinkle care with Niacinamide and Adenosine",
                 "Quick absorption with a fresh, non-sticky finish, suitable for all skin types"]}}
    '''

    INPUT : {input}
    FORMAT : {format}
    TYPES OF PRODUCTS : {types_of_products}

    """
)

plantUML_extract_prompt_bldata= plantUML_extract_prompt_bldata.partial(keyword_num=keyword_num)
plantUML_extract_prompt_bldata= plantUML_extract_prompt_bldata.partial(format=parser.get_format_instructions())
plantUML_extract_prompt_bldata= plantUML_extract_prompt_bldata.partial(types_of_products="""[Textured Protein Substances, Protein Hydrolysates, Non-Alcoholic Beverage Mixes,
                                                                                              Alcoholic Beverage Mixes, Flavored Powders, Sugar-Based Sweeteners, Herbal Infusions,
                                                                                             Dietary Supplements, Flavoring Granules]""")


plantUML_extract_chain_bldata = plantUML_extract_prompt_bldata | chat_openai | parser

NameError: name 'plantuml_extract_prompt_bldata' is not defined

In [ ]:
bldata_preprocessed_dict_openai_ext=joblib.load('bldata_preprocessed_dict_openai.pkl')

In [ ]:
for i, (key, value) in tqdm(enumerate(bldata_preprocessed_dict_openai_ext.items())):
  try:
    response_bldata = plantUML_extract_chain_bldata.invoke({"input": value[0]})
    if i%100==0:
      print(response_bldata.type)
    bldata_preprocessed_dict_openai_ext[key].append(response_bldata.name)
    bldata_preprocessed_dict_openai_ext[key].append(response_bldata.type)
    bldata_preprocessed_dict_openai_ext[key].append(response_bldata.keyword)
    bldata_preprocessed_dict_openai_ext[key].append(response_bldata.phrase)
  except Exception as e:
    print(f"key : {key}")
    print(f"error type : {e}")
    print(f"extract error: {response_bldata}")

1it [00:02,  2.31s/it]

Not Classifiable


101it [02:57,  2.37s/it]

Not Classifiable


201it [05:46,  1.53s/it]

...

key : HS CODE 2106 9098 INVOICE 202009704 US F UP BELGIAN WAFFLES MIX 600G US F UP FRENCH CREPE<br/>
error type : Failed to parse DescriptionSummary from completion null. Got: 1 validation error for DescriptionSummary
  Input should be a valid dictionary or instance of DescriptionSummary [type=model_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.10/v/model_type
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
extract error: name='DIAGEO Liquors Shipment' type='Not Classifiable' keyword=['Importation', 'Logistics', 'Alcoholic beverages', 'Freight prepaid', 'Regulatory compliance'] phrase=['Shipment of alcoholic beverages imported from DIAGEO OPERATIONS ITALY S.P.A. to DIAGEO NORTH AMERICA INC.', "Shipping costs covered by the supplier indicated by 'freight prepaid'", 'Includes various identifiers for tracking and regulatory purposes']


401it [18:27,  7.24s/it]

Sugar-Based Sweeteners


478it [29:42,  3.73s/it]


KeyboardInterrupt: 

In [ ]:
bldata_plantuml_openai=pd.DataFrame.from_dict(bldata_preprocessed_dict_openai_ext, orient='index', columns=['type'])
bldata_plantuml_openai.reset_index(inplace=True)
bldata_plantuml_openai.rename(columns={'index':'product_name'}, inplace=True)
bldata_plantuml_openai.to_csv('bldata_plantuml_openai_v3.csv', index=False)

display(bldata_plantuml_openai)

,product_name,type
0,2700 BOXES FROZEN STRAWBERRY (FRESA CONGELADA)...,Not Classifiable
1,FOOD PREPARATION NAMKEENS/PUFF S IEC NO. : 059...,snacks
2,CARGO STOWED IN A REFRIGER ATED CONTAINER AT M...,Not Classifiable
3,"SHIPPER S LOAD, COUNT, SEAL AND WEIGHT 2X20 CY...",Not Classifiable
4,416PKGS S.T.C. 414CTNS + 2WDCS FOOD & UTENSI L...,Not Classifiable
...,...,...
2821,560 CASES FROZEN IQF BERRY MEDLEY NO STRAWBERR...,[This product is a frozen IQF (Individually Qu...
2822,FOOD PREPARATION NAMKEENS/SWEE TS IEC NO. : 05...,"[This product is a food preparation known as ""..."
2823,HS CODE 21069099 AUT HENTIC INDIAN SNACK S 3 S...,"[This product is called ""Authentic Indian Snac..."
2824,900 CASE(S) 000000004404011038 - BACO 12X75 JP...,[The product description provided appears to b...
